决策树（decision tree）是一种基本的分类与回归方法，可以看做是一系列 if-then 规则的集合，可读性强，速度快。本章主要讨论用于分类的决策树，重点掌握特征选择、决策树生成和剪枝三个部分

* [5.1 决策树模型与学习](#5.1-决策树模型与学习)
  * [5.1.1 决策树模型](#5.1.1-决策树模型)
  * [5.1.2 决策树与 if-then 规则](#5.1.2-决策树与-if\-then-规则)
  * [5.1.3 决策树与条件概率分布](#5.1.3-决策树与条件概率分布)
  * [5.1.4 决策树学习](#5.1.4-决策树学习)
* [5.2 特征选择](#5.2-特征选择)
  * [5.2.1 特征选择的问题](#5.2.1-特征选择的问题)
  * [5.2.2 信息增益](#5.2.2-信息增益)
  * [5.2.3 信息增益比](#5.2.3-信息增益比)
* [5.3 决策树的生成](#5.3-决策树的生成)
  * [5.3.1 ID3算法](#5.3.1-ID3算法)
  * [5.3.2 C4.5 算法](#5.3.2-C4.5-算法)
* [5.4 决策树的剪枝](#5.4-决策树的剪枝)
* [5.5 CART 算法](#5.5-CART-算法)
  * [5.5.1 CART 生成](#5.5.1-CART-生成)
  * [5.5.2 CART 剪枝](#5.5.2-CART-剪枝)
* [算法实现](#算法实现)
* [习题](#习题)



# 5.1 决策树模型与学习

## 5.1.1 决策树模型

决策树的每个内部结点（internal node）表示一个特征或属性，叶结点（leaf node）表示一个类。

每次从根结点开始，对某项特征进行测试，根据结果分配到子结点，每个子结点对应一个特征取值，递归直到抵达某个叶结点。

## 5.1.2 决策树与 if-then 规则

一棵决策树可以视为 if-then 规则的集合。从根结点到叶结点每一条路径构建一条规则，内部结点表示条件，叶结点代表结论。

这些路径都是互斥且完备的。即每个实例由且仅由一条路径确定。

## 5.1.3 决策树与条件概率分布

决策树还可以表示为给定特征条件下的条件概率分布。这一条件概率分布构成了一个特征空间的划分（partition），将特征空间划分为互不相交的若干单元（cell）或区域（region）。

各叶结点（单元）上的条件概率通常表现为某一个类别的概率较大，决策树分类时会将该结点的实例强行归为条件概率大的那一类中去。

##  5.1.4 决策树学习

决策树学习本质上是要得到一套分类规则，既能很好地拟合训练集，也能很好地预测未知数据。满足条件的决策树可能有多棵，也可能不存在。

决策树的损失函数通常是正则化的极大似然函数，学习策略就是极小化损失函数。从所有可能的树中找最优是 NP 完全问题，通常选择启发式方法，近似求解这一问题，得到一个次最优（sub-optimal）的。

学习开始时，所有数据都在根结点，选择一个最优特征，将数据分割。如果已经基本分类正确，则确认构建根结点，并将数据分配到子结点去；如果还有未正确分类的数据，则对该子集继续选择最优特征进行分割。如此递归，直到都被正确分类或没有合适的特征了，每个子集都被分到一个叶结点上，决策树生成完成。

上述过程完成后，需要自下而上的剪枝以防止过拟合。将部分结点回退至父结点乃至祖先结点，使其成为新的叶结点。

同时因为决策树对应着条件概率分布，深浅不同的决策树概率复杂度也就不同。决策树的生成考虑局部最优，剪枝要考虑全局最优。

# 5.2 特征选择

## 5.2.1 特征选择的问题

特征选择在于选取具有分类能力的特征，如果使用一个特征分类后和随机分类差别不大，则认为该特征不具备分类能力。经验上可以丢弃该特征。

而面对某个数据集，常常有不同的特征都可以划分特征空间，那么就需要选择、比较特征的准则了，常用准则是信息增益或信息增益比

## 5.2.2 信息增益

信息论中用熵（entropy）表示随机变量不确定性的大小，或者说消除不确定性需要知道的信息量。随机变量 $X$ 的熵与 $X$ 的取值无关，只与 $X$ 的分布有关，定义为：
$$
H(X)=H(p)=-\sum_{i=1}^{n} p_{i} \log p_{i}
$$
如果 $p_{i} = 0$，则定义 $0 \log 0 = 0$

由定义可知 $0 \leqslant H(p) \leqslant \log n$，对数的底可以是 2 或自然常数 e ，此时熵的单位分别为比特（bit）和纳特（nat）。

如果 $X$ 是 0-1 分布，则其熵为：
$$
H(p)=-p \log _{2} p-(1-p) \log _{2}(1-p)
$$
对应图像如下，$p=0$ 或 $p=1$ 时是确定事件，没有不确定性， $p = 0.5$ 时不确定性最大，消除不确定性需要的信息最多

![]( https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-08-28_entropy.png)

而对随机变量 $(X,Y)$，其联合概率分布
$$
P\left(X=x_{i}, Y=y_{j}\right)=p_{i j}, \quad i=1,2, \cdots, n ; \quad j=1,2, \cdots, m
$$
条件熵 $H(Y|X)$ 表示在知道 $X$ 的情况下，想要明确 $Y$ 所需的信息量，或者说此时 $Y$ 的不确定性大小，定义为：
$$
H(Y | X)=\sum_{i=1}^{n} p_{i} H\left(Y | X=x_{i}\right)
$$
如果有 0 概率，则令 $0 \log 0 = 0$

当熵和条件熵中的概率由数据估计（特别是极大似然估计）得到时，对应的熵分别称为经验熵（empirical entropy）和经验条件熵（empirical conditional entropy）

信息增益（information gain）表示知道 $X$ 后 $Y$ 的不确定性消减了多少，具体地，特征 $A$ 对数据集 $D$ 的信息增益 $g(D,A)$ 定义为：
$$
g(D, A)=H(D)-H(D | A)
$$
熵与条件熵的差称为互信息（mutual information），决策树中的信息增益等价于特征和类别的互信息。

不同特征信息增益不同，每次选择信息增益最大的特征，具体地：

1. 计算经验熵 $H(D)$:

$$
H(D)=-\sum_{k=1}^{K} \frac{\left|C_{k}\right|}{|D|} \log _{2} \frac{\left|C_{k}\right|}{|D|}
$$

$\left|C_{k}\right|$ 为属于类别 $C_{k}$ 的样本个数，$\left|D\right|$ 表示数据集样本容量

2. 计算经验条件熵 $H(D|A)$:

$$
H(D | A)=\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{|D|} H\left(D_{i}\right)=-\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{|D|} \sum_{k=1}^{K} \frac{\left|D_{i k}\right|}{\left|D_{i}\right|} \log _{2} \frac{\left|D_{i k}\right|}{\left|D_{i}\right|}
$$

特征 $A$ 将数据集 $D$ 分为 $n$ 个子集，$D_{i}$ 表示第 $i$ 个子集，$\left|D_{i}\right|$ 表示其样包含样本数，$\left|D_{ik}\right|$ 表示第 $i$ 个子集中第 $k$ 类的样本数

3. 计算信息增益

## 5.2.3 信息增益比

信息增益作为选择标准，会偏袒取值选择较多的特征。因为极限情况下如果 $\left|D_{i}\right| = 1$，$H(D|A) = 0$。为此，引入信息增益比（information gain ratio）作为另一选择标准。定义为：
$$
g_{R}(D, A)=\frac{g(D, A)}{H_{A}(D)}
$$
其中 $H_{A}(D)=-\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{|D|} \log _{2} \frac{\left|D_{i}\right|}{|D|}$，表示特征 $A$ 分割集合 $D$ 后取得某一子集时面对的不确定性。

# 5.3 决策树的生成

## 5.3.1 ID3算法

ID3 算法每次在各结点上使用信息增益选择特征，并按照信息增益最大的特征构建子结点，递归直到各个特征的增益小于给定阈值或没有特征可选为止。

ID3 算法相当于使用极大似然法进行概率模型选择。因为只有树的生成，很容易过拟合

## 5.3.2 C4.5 算法

C4.5 算法是对 ID3 的改进，选用信息增益比作为特征选择标准，避免了特征选择时的倾向性。

输入：训练数据集 $D$，特征集 $A$ 与阈值 $\epsilon$

输出：决策树 T

1. 如果当前 $D$ 内实例都属于同一类别 $C_{k}$，返回 $T$

2. 如果 $A = \varnothing$ ，取 $D$ 中实例数最大类别 $C_{k}$，返回 $T$
3. 计算信息增益（比），选择最大特征 $A_{g}$
4. 如果 $A_{g}$ 的信息增益（比）小于阈值 $\epsilon$，取 $D$ 中实例数最大类别 $C_{k}$ ，返回 $T$
5. 否则，依照 $A_{g}$ 的每个取值 $a_{i}$ 分割 $D$ 为若干非空子集 $D_{i}$，取各自实例数最大类别并构建子结点，返回 $T$
6. 选择 $D_{i}$ 为新的训练集，新特征集为 $A - \{A_{g}\}$，递归得到子树 $T_{i}$，返回 $T_{i}$

# 5.4 决策树的剪枝

整个决策树生成过程都在极尽全力尝试对训练数据正确分类，这往往导致过拟合，模型复杂度过高。需要适度简化模型，即剪枝（pruning）。

剪枝就是裁掉些子树或叶子节点，使父节点作为新叶子节点。具体的，可以通过极小化整棵决策树的损失函数来实现。损失函数定义为：
$$
C_{\alpha}(T)=C(T)+\alpha|T|
$$
其中 $|T|$ 为树的叶节点数，也代表模型复杂度，$C(T)$ 表示模型对训练数据的预测误差，$\alpha \geqslant 0$ 是调控二者影响的参数。$C(T)$ 计算方式为：
$$
C(T)=\sum_{t=1}^{|T|} N_{t} H_{t}(T)
$$
$t$ 是 $T$ 的叶节点，该节点下的实例数为 $N_{t}$，归属于第 $k$ 类的个数为 $N_{tk}$，$H_{t}(T)$ 表示叶节点 $t$ 的经验熵：
$$
H_{t}(T)=-\sum_{k} \frac{N_{t k}}{N_{t}} \log \frac{N_{t k}}{N_{t}} \\
C(T)=-\sum_{t=1}^{|T|} \sum_{k=1}^{K} N_{t k} \log \frac{N_{t k}}{N_{t}}
$$
可见，剪枝就是给定 $\alpha$ 值情况下，选择损失函数最小的模型。而根据定义，极小化损失函数等价于正则化的极大似然估计

输入：决策树 $T$，参数 $\alpha$

输出：修剪后的子树 $T_{\alpha}$

1. 计算结点经验熵
2. 递归地从叶结点向上收缩
3. 收缩前后两棵树为 $T_{A}$ ，$T_{B}$。如果 $C_{\alpha}\left(T_{B}\right) \leqslant C_{\alpha}\left(T_{A}\right)$，执行剪枝
4. 重复上述步骤直至不能继续为止

注意到前后损失函数差值计算可以只在局部进行，所以可以通过动态规划算法实现。

# 5.5 CART 算法

分类与回归树（classification and regression tree，CART）同样由特征选择、树的生成和剪枝构成，既可以用于分类也可以用于回归。

CART 是对给定输入计算输出的条件概率分布的学习方法，是一棵二叉树，将每个结点分为“是“和”否”两支，将特征空间划分为若干单元，在这些单元上计算预测概率。

CART 算法分两步：

1. 根据训练集生成决策树，生成的树尽量大
2. 根据验证集剪枝，以损失函数最小为标准

## 5.5.1 CART 生成

生成树靠递归。回归树使用平方误差最小准则，分类树使用基尼指数（Gini index）最小化准则

* **回归树的生成**

输入：数据集 $D$

输出：回归树 $f(x)$

将输入空间递归地将每个区域划分为两个子区域，并决定每个子区域上的输出值

① 选择最优切分变量 $j$ 与切分点 $s$，即选用 $x^{(j)}$ 及其取值划分区域。求解
$$
\min _{j, s}\left[\min _{c_{1}} \sum_{x_{i} \in R_{1}(j, s)}\left(y_{i}-c_{1}\right)^{2}+\min _{c_{2}} \sum_{x_{i} \in R_{2}(j, s)}\left(y_{i}-c_{2}\right)^{2}\right]
$$
$R_{1}$（$x^{(j)}\leqslant s$） 和 $R_{2}$  （$x^{(j)}> s$） 是划分后的两个区域，两个区域的输出值为 $c_{1}$ 和 $c_{2}$。

对于固定地变量 $j$ 可以找到最优切分点 $s$ 使 $\hat{c}_{1}=\operatorname{ave}\left(y_{i} | x_{i} \in R_{1}(j, s)\right)$ 和 $\hat{c}_{2}=\operatorname{ave}\left(y_{i} | x_{i} \in R_{2}(j, s)\right)$

遍历变量 $j$ 扫描切分点 $s$，找到使上式最小的数对 $(j,s)$

② 用选定的 $(j,s)$ 划分区域决定输出值：
$$
\begin{array}{c}{R_{1}(j, s)=\left\{x | x^{(j)} \leqslant s\right\}, \quad R_{2}(j, s)=\left\{x | x^{(j)}>s\right\}} \\ {\hat{c}_{m}=\frac{1}{N_{m}} \sum_{x_{i} \in R_{m}(j, s)} y_{i}, \quad x \in R_{m}, \quad m=1,2}\end{array}
$$
③ 对两个子区域重复 ①、②两步，直至满足停止条件

④ 输入空间被划分为 $M$ 个区域 $R_{1},R_{2},\ldots,R_{M}$，得到决策树：
$$
f(x)=\sum_{m=1}^{M} \hat{c}_{m} I\left(x \in R_{m}\right)
$$
停止条件可以是区域中样本个数小于预定阈值，或是累积误差小于预定阈值，或者没有更多特征

* **分类树的生成**

什么是基尼指数呢？分类问题中，假如共有 $K$ 个类别，样本点属于第 $k$ 类的概率为 $p_{k}$，概率分布的基尼指数定义为
$$
\operatorname{Gini}(p)=\sum_{k=1}^{K} p_{k}\left(1-p_{k}\right)=1-\sum_{k=1}^{K} p_{k}^{2}
$$
对于二分类问题，如果样本点属于第一个类别的概率为 $p$，则概率分布的基尼指数为
$$
\operatorname{Gini}(p) = 2p(1-p)
$$
给定样本集合 $D$，其基尼指数为
$$
\operatorname{Gini}(D)=1-\sum_{k=1}^{K}\left(\frac{\left|C_{k}\right|}{|D|}\right)^{2}
$$
$C_{k}$ 是属于第 $k$ 类的样本子集

输入：训练集，停止计算条件

输出：CART 决策树

① 对每一个特征的每一个取值计算基尼指数，比如依照特征 $A = a$ 将数据集 $D$ 分为了 $D_{1},D_{2}$，基尼指数为
$$
\operatorname{Gini}(D, A)=\frac{\left|D_{1}\right|}{|D|} \operatorname{Gini}\left(D_{1}\right)+\frac{\left|D_{2}\right|}{|D|} \operatorname{Gini}\left(D_{2}\right)
$$
② 从所有基尼指数中选最小值对应的特征及其切分点作为最优特征和最优切分点。将数据集按该切分划归到两个子结点去

③ 递归调用 ①、② 直至满足停止条件

④ 生成 CART 决策树

停止条件可以是区域中样本个数小于预定阈值，或是样本集基尼指数小于给定阈值，或者没有更多特征

基尼指数表示集合 $D$ 的不确定性，和熵类似，实际上对于二分类问题，基尼指数 Gini(p)，熵之半 $\frac {H(p)}{2}$ 和分类误差率关系如下

![]( https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-09-02_gini_entropy.png)

## 5.5.2 CART 剪枝

CART 剪枝是将一棵“生长完全”的树不断剪枝，得到一系列子树，再通过交叉验证法选取最优子树

首先是剪枝，子树 $T$ 的损失函数 $C_{\alpha}(T)=C(T)+\alpha|T|$，当 $\alpha$ 确定时，最优子树 $T_{\alpha}$ 存在且唯一，此时 $C_{\alpha}(T)$ 最小。而 $\alpha$ 是权衡局部预测误差和整体复杂度的，当 $\alpha$ 较小，树就倾向于大且深，极端时 $\alpha = 0$ ，此时是整体树，反之，当 $\alpha$ 较大，树倾向于小且浅，极端时 $\alpha = +\infty$，此时是只有根结点的单结点树。

对于内部结点 $t$ ,损失为 $C_{\alpha}(t)=C(t)+\alpha$，以其为根的子树 $T_{t}$ 损失函数为 $C_{\alpha}\left(T_{t}\right)=C\left(T_{t}\right)+\alpha\left|T_{t}\right|$

从上文推断来看，随着 $\alpha$ 取值由 0 到 $\infty$ ，伴随着 $C_{\alpha}(t) > C_{\alpha}(T_{t}),\quad C_{\alpha}(t)=C_{\alpha}(T_{t}),\quad C_{\alpha}(t)<C_{\alpha}(T_{t})$ 几段变化

而 $C_{\alpha}(t) = C_{\alpha}(T_{t})$ 就是剪枝目标，剪枝前后整体损失变化可以表示为
$$
g(t)=\alpha=\frac{C(t)-C\left(T_{t}\right)}{\left|T_{t}\right|-1}
$$
剪去 $g(t)$ 最小的子树 $T_{i}$， 更新 $\alpha_{i + 1} = \min g(t)$，递归向上修剪，直到根节点。最终得到一系列子树 $T_{0},T_{1},\ldots,T_{n}$

输入：CART 算法生成的决策树 $T_{0}$

输出：最优决策树 $T_{\alpha}$

① 设 $k = 0, T = T_{0}$

② 设 $\alpha = +\infty$

③ 自下而上计算各内部结点的 $C\left(T_{t}\right),\left|T_{t}\right|,g(t)$ 并取 $\alpha = \min (\alpha, g(t))$ 

④ 对 $g(t)=\alpha$ 的内部结点进行剪枝，对该内部结点 $t$ 进行多数表决确定类别，得到树 $T$

⑤ 更新 $k = k + 1, \alpha_{k} = \alpha, T_{k} = T$

⑥ 如果 $T_{k}$ 不是由根结点与两个叶结点构成，返回 ②；否则令 $T_{k} = T_{n}$

⑦ 使用交叉验证法在子树序列 $T_{0},T_{1},\ldots,T_{n}$ 中选择最优子树 $T_{\alpha}$



# 算法实现

**导入相关库**

In [1]:
import numpy as np
import pandas as pd
import math

**硬件与版本信息**

In [2]:
%load_ext watermark
%watermark -v -m -p ipywidgets,matplotlib,numpy,pandas,sklearn

CPython 3.7.3
IPython 7.6.1

ipywidgets 7.5.0
matplotlib 3.1.0
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit


**数据结构分析与构建**

我们选取最常见的二叉树来实现决策树，回归树损失判定是方差缩减，分类时是信息增益。我们不妨比对一下与白板编程构建二叉树时的异同

通常我们会先定义一个结点类 `Node`，成员包括结点存储的值`value`，指向左子树和右子树根节点的`left_child`和`right_child`

决策树中我们一样需要一个结点类`DNode`，结点的输出`value`分类树下是预测的某个类别，回归树下是一个浮点数，左右指针成了“是”`yes_subtree`与“否”`no_subtree`，此外还多了两个成员，一个判定特征取值，决定左右走向的阈值`thredhold`，另一个是特征索引`feature_i`

In [3]:
class DNode:
    """叶结点或内部结点"""
    
    def __init__(self, feature_i=None, threshold=None, value=None,
                 yes_subtree=None, no_subtree=None):
        self.feature_i = feature_i
        self.threshold = threshold
        self.value = value
        self.yes_subtree = yes_subtree
        self.no_subtree = no_subtree   

辅助函数设计

In [4]:
def divide_on_feature(X, feature_i, threshold):
    '''对选定特征进一步分类，获取独热数据集'''
    split_func = None
    if isinstance(threshold, int) or isinstance(threshold, float):
        split_func = lambda sample:sample[feature_i] >= threshold
    else:
        split_func = lambda sample:sample[feature_i] == threshold
    
    X_1 = np.array([sample for sample in X if split_func(sample)])
    X_2 = np.array([sample for sample in X if not split_func(sample)])
    
    return np.array([X_1, X_2])

In [5]:
class DecisionTree(object):
    """回归树和分类树的父类"""
    
    def __init__(self, min_split_num=2, min_impurity=1e-7):
        
        # 决策树根节点
        self.root = None
        
        # 最小切分单位大小，样本数少于该值不进行进一步分割
        self.min_split_num = min_split_num
        
        # 最小增益值，当划分带来的增益小于该值时停止生成
        self.min_impurity = min_impurity
        
        # 计算增益，分类树下计算信息增益，回归树下计算方差缩减程度
        self.__cal_impurity = None
        
        # 计算叶子结点给出的预测 y
        self.__cal_leaf_val = None
        
        # y 是否经过 one-hot 编码，默认没有(one-dim)
        self.one_dim = None
        
    def fit(self, X_train, y_train):
        self.one_dim = len(np.shape(y_train)) == 1
        self.root = self.__build_tree(X_train, y_train)
    
    def __build_tree(self, X_train, y_train):
        """递归创建一棵决策树
        
        对 X 按特征进行划分，比对不同划分选择下的误差以进行最优分类
        """
        max_impurity = 0
        best_criteria = None  # 最优划分特征
        best_sets = None  # 最优划分形成的子集集合
        
        # 拼接成习惯的训练集形式
        if len(np.shape(y_train)) == 1:
            y_train = np.expand_dims(y_train, axis=1)
        train_set = np.concatenate((X_train, y_train),axis=1)
        
        n_samples, n_features = np.shape(X_train)
        
        if n_samples > self.min_split_num: # 停止条件之一
            
            for feature_i in range(n_features):
                
                # 当前特征可取哪些值
                feature_values = np.expand_dims(X_train[:,feature_i], axis=1)
                unique_values = np.unique(feature_values)
                
                # 计算以当前特征为划分标准时对应的误差
                for threshold in unique_values:
                    
                    X_y1, X_y2 = divide_on_feature(train_set, feature_i, threshold)
                    
                    if len(X_y1) > 0 and len(X_y2) > 0:  # 还有划分的必要
                        y1 = X_y1[:, n_features:]
                        y2 = X_y2[:, n_features:]
                        
                        impurity = self.__cal_impurity(y_train, y1, y2)
                        
                        if impurity > max_impurity:
                            max_impurity = impurity
                            best_criteria = {"feature_i":feature_i, "threshold":threshold}
                            best_sets = {
                                "yes_X": X_y1[:, : n_features],
                                "yes_y": X_y1[:, n_features : ],
                                "no_X": X_y2[:, : n_features],
                                "no_y": X_y2[:, n_features : ]
                            }
                            
                # 还值得继续生成树
                if max_impurity > self.min_impurity:
                
                    yes_subtree = self.__build_tree(best_sets["yes_X"],best_sets["yes_y"])
                    no_subtree = self.__build_tree(best_sets["no_X"],best_sets["no_y"])
                    return DNode(feature_i=best_criteria["feature_i"], threshold=best_criteria["threshold"],
                            yes_subtree=yes_subtree, no_subtree=no_subtree)
        
        # 停止划分，已经成为叶子结点了，计算此时的预测输出
        leaf_value = self.__cal_leaf_val(y_train)
            
        return DNode(value=aleaf_value)
    
    def predict_value(self, x, tree=None):
        '''树tree对样本x的预测,递归实现'''
        
        if tree is None:
            tree = self.root
        
        # 抵达叶子结点，给出预测
        if tree.value is not None:
            return tree.value
        
        # 还在内部结点，选择前进方向
        feature_value = x[tree.feature_i]
        
        goto = tree.yes_subtree
        if isinstance(feature_value, int) or isinstance(feature_valuee, float):
            if feature_value < tree.threshold:
                goto = tree.no_subtree
        elif feature_valuee != threshold:
            goto = tree.no_subtree
        
        return self.predict_value(x, goto)
    
    def predict(self, X_test):
        y_pred = [self.predict_value(sample) for sample in X_test]
        return y_pred

* 分类树

辅助函数设计

In [6]:
def cal_entropy(y):
    unique_labels = np.unique(y)
    entropy = 0
    for label in unique_labels:
        count = len(y[y == label])
        p = count / len(y)
        entropy += -p * math.log2(p)
    return entropy

In [7]:
class ClassifyDT(DecisionTree):
    
    def __cal_info_gain(self, y_train, y1, y2):
        
        p = len(y1) / len(y_train)
        entropy = cal_entropy(y_train)
        info_gain = entropy - p * cal_entropy(y1) - (1 - p) * cal_entropy(y2)
        
        return info_gain
    
    def __majority_vote(self, y):
        most = None
        max_count = 0
        
        for label in np.unique(y):
            count = len(y[y == label])
            if count > max_count:
                most = label
                max_count = count
        return most
    
    def fit(self, X_train, y_train):
        self.__cal_impurity = self.__cal_info_gain
        self.__cal_leaf_val = self.__majority_vote
        super(ClassifyDT, self).fit(X_train, y_train)

* 回归树

辅助函数设计

In [8]:
def cal_var(X):
    mean = np.ones(np.shape(X)) * X.mean(0)
    n_samples = np.shape(X)[0]
    var = (1 / n_samples) * np.diag((X-mean)).T.dot(X - mean)
    
    return var

In [9]:
class RegressDT(DecisionTree):
    
    def __cal_var_reduct(self, y_train, y1, y2):
        
        var_tot = cal_var(y_train)
        var_1 = cal_var(y1)
        var_2 = cal_var(y2)
        
        frac_1 = len(y1) / len(y)
        frac_2 = len(y2) / len(y)
        
        var_reduct = var_tot - (frac_1 * var_1 + frac_2 * var_2)
        
        return sum(var_reduct)
    
    def __mean(self, y):
        value = np.mean(y, axis=0)
        return value if len(value) > 1 else value[0]
    
    def fit(self, X_train, y_train):
        self.__cal_impurity = self.__cal_var_reduct
        self.__cal_leaf_val = self.__mean
        super(RegressDT, self).fit(X_train, y_train)

# 习题

**5.1 根据表5.1所给的训练数据集，利用信息增益比（C4.5算法）生成决策树。**

<img src=" https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-09-04_table_5_1.png" style="zoom:75%;" />

解：

令 $A_{1} = \{1, 2, 3\},A_{2} = \{1,2\},A_{3}=\{1,2\},A_{4}=\{1,2,3\}$，分别代表 $年龄:\{青年,中年,老年\}$，$有工作:\{是,否\}$，$有自己的房子:\{是,否\}$，$信贷情况:\{一般,好,非常好\}$。

首先，计算经验熵 $H(D)$
$$
H(D)=-\frac{9}{15} \log _{2} \frac{9}{15}-\frac{6}{15} \log _{2} \frac{6}{15}=0.971
$$
计算四个特征的信息增益
$$
\begin{aligned} 
g\left(D, A_{1}\right)=& 0.971-\left[\frac{5}{15}\left(-\frac{2}{5} \log _{2} \frac{2}{5}-\frac{3}{5} \log _{2} \frac{3}{5}\right)+\right.\\ 
&\left.\frac{5}{15}\left(-\frac{3}{5} \log _{2} \frac{3}{5}-\frac{2}{5} \log _{2} \frac{2}{5}\right)+\frac{5}{15}\left(-\frac{4}{5} \log _{2} \frac{4}{5}-\frac{1}{5} \log _{2} \frac{1}{5}\right)\right] \\
=& 0.971-0.888=0.083 \\
g\left(D, A_{2}\right)=& 0.971-\left[\frac{5}{15} \times 0+\frac{10}{15}\left(-\frac{4}{10} \log _{2} \frac{4}{10}-\frac{6}{10} \log _{2} \frac{6}{10}\right)\right]=0.971-0.647 = 0.324 \\
g\left(D, A_{3}\right) =&0.971-\left[\frac{6}{15} \times 0+\frac{9}{15}\left(-\frac{3}{9} \log _{2} \frac{3}{9}-\frac{6}{9} \log _{2} \frac{6}{9}\right)\right] =0.971-0.551=0.420 \\
g\left(D, A_{4}\right)=& 0.971-\left[\frac{5}{15}\left(-\frac{1}{5} \log _{2} \frac{1}{5}-\frac{4}{5} \log _{2} \frac{4}{5}\right)+\right.\\ 
&\left.\frac{4}{15}\times 0+\frac{6}{15}\left(-\frac{4}{6} \log _{2} \frac{4}{6}-\frac{2}{6} \log _{2} \frac{2}{6}\right)\right] \\
=& 0.971-0.608=0.363
\end{aligned}
$$
计算数据集 $D$ 相对四个特征的熵 $H_{A}(D)$
$$
\begin{aligned}
H_{A_{1}}(D) &= -\frac{1}{3}\log \frac{1}{3} \times 3=1.585 \\
H_{A_{2}}(D) &= -\frac{1}{3}\log \frac{1}{3}-\frac{2}{3}\log \frac{2}{3}=0.918 \\
H_{A_{3}}(D) &= -\frac{6}{15}\log \frac{6}{15}-\frac{9}{15}\log \frac{9}{15}=0.971 \\
H_{A_{4}}(D) &= -\frac{5}{15}\log \frac{5}{15}-\frac{6}{15}\log \frac{6}{15}-\frac{4}{15}\log \frac{4}{15}=1.566
\end{aligned}
$$
得到四个特征对应的信息增益比
$$
\begin{aligned}
g_{R}(D,A_{1}) &= \frac{g(D,A_{1})}{H_{A_{1}}(D)} = 0.052\\
g_{R}(D,A_{2}) &= \frac{g(D,A_{2})}{H_{A_{2}}(D)} = 0.353 \\
g_{R}(D,A_{3}) &= \frac{g(D,A_{3})}{H_{A_{3}}(D)} = 0.433 \\
g_{R}(D,A_{4}) &= \frac{g(D,A_{4})}{H_{A_{4}}(D)} = 0.234
\end{aligned}
$$
选择 `是否有房` 为第一特征，分割成两个新数据集 $D_{1},D_{2}$

| 年龄 | 有工作 | 信贷情况 | 类别 |
| ---- | ------ | -------- | ---- |
| 青年 | 是     | 一般     | 是   |
| 中年 | 是     | 好       | 是   |
| 中年 | 否     | 非常好   | 是   |
| 中年 | 否     | 非常好   | 是   |
| 老年 | 否     | 非常好   | 是   |
| 老年 | 否     | 好       | 是   |

| 年龄 | 有工作 | 信贷情况 | 类别 |
| ---- | ------ | -------- | ---- |
| 青年 | 否     | 一般     | 否   |
| 青年 | 否     | 好       | 否   |
| 青年 | 是     | 好       | 是   |
| 青年 | 否     | 一般     | 否   |
| 中年 | 否     | 一般     | 否   |
| 中年 | 否     | 好       | 否   |
| 老年 | 是     | 好       | 是   |
| 老年 | 是     | 非常好   | 是   |
| 老年 | 否     | 一般     | 否   |

$D_{1}$ 已完成分类，继续对 $D_{2}$ 进行划分，此时的经验熵 $H(D_{2})$
$$
H(D_{2}) = -\frac{3}{9}\log \frac{3}{9}-\frac{6}{9}\log \frac{6}{9}=0.918
$$
三个特征的信息增益为
$$
\begin{aligned}
g_{R}(D_{2},A_{1}) &= 0.918 -\left[\frac{2}{9} \times 0+\frac{4}{9}\left(-\frac{1}{4} \log _{2} \frac{1}{4}-\frac{3}{4} \log _{2} \frac{3}{4}\right)+\frac{3}{9}\left(-\frac{1}{3} \log _{2} \frac{1}{3}-\frac{2}{3} \log _{2} \frac{2}{3}\right)\right] \\
&=0.918-0.667 = 0.251 \\
g_{R}(D_{2},A_{2}) &= 0.918-\left[\frac{3}{9}\times0+\frac{6}{9} \times 0 \right]
=0.918 \\
g_{R}(D_{2},A_{4}) &= 0.918-\left[\frac{4}{9} \times 0+\frac{4}{9}\left(-\frac{2}{4} \log _{2} \frac{2}{4}-\frac{2}{4} \log _{2} \frac{2}{4}\right)+\frac{1}{9}\times 0 \right] 
=0.918-0.444 = 0.474
\end{aligned}
$$
数据集 $D_{2}$ 关于三个特征的熵为
$$
\begin{aligned}
H_{A_{1}}(D_{2}) &= -\frac{4}{9}\log \frac{4}{9}-\frac{2}{9}\log \frac{2}{9}-\frac{3}{9}\log \frac{3}{9}=1.531 \\
H_{A_{2}}(D_{2}) &= -\frac{3}{9}\log \frac{3}{9}-\frac{6}{9}\log \frac{6}{9}=0.918\\
H_{A_{4}}(D_{2}) &= -\frac{4}{9}\log \frac{4}{9}-\frac{4}{9}\log \frac{4}{9}-\frac{1}{9}\log \frac{1}{9}=1.392
\end{aligned}
$$
三个特征的信息增益比为
$$
\begin{aligned}
g_{R}(D_{2},A_{1}) &= \frac{g(D_{2},A_{1})}{H_{A_{1}}(D_{2})} = 0.164\\
g_{R}(D_{2},A_{2}) &= \frac{g(D_{2},A_{2})}{H_{A_{2}}(D_{2})} = 1 \\
g_{R}(D_{2},A_{4}) &= \frac{g(D_{2},A_{4})}{H_{A_{4}}(D_{2})} = 0.341 
\end{aligned}
$$
选择 `是否有工作` 为第二特征，划分 $D_{2}$，为新的数据集 $D_{3},D_{4}$，新数据集都已完成分类，决策树生成完毕

![]( https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-09-05_fin_decision_tree.png)

**5.2 试用平方误差准则生成一个二叉回归树**

假设有如下数据集 $X$，$x \in [0.5, \quad 10.5]$，输出 $y \in [5.0, \quad 10.0]$，停止标准为区域误差 $\theta \leqslant 0.05$ 或是已无特征可分

| 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| 5.56 | 5.70 | 5.91 | 6.40 | 6.80 | 7.05 | 8.90 | 8.70 | 9.00 | 9.05 |

我们要找到划分点 $s$，得到两个划分：
$$
R_{1}=\{x | x \leqslant s\}, \quad R_{2}=\{x | x>s\}
$$
选择划分变量 $j$ ，和划分点一起构建数对 $(j, s)$ 使得
$$
m(s)=\min _{j, s}\left[\min _{c_{1}} \sum_{x_{i} \in R_{1}(j, s)}\left(y_{i}-c_{1}\right)^{2}+\min _{c_{2}} \sum_{x_{i} \in R_{2}(j, s)}\left(y_{i}-c_{2}\right)^{2}\right] \\
c_{1}=\frac{1}{N_{1}} \sum_{x_{i} \in R_{1}} y_{i}, \quad c_{2}=\frac{1}{N_{2}} \sum_{x_{i} \in R_{2}} y_{i}
$$
本例中只有一个维度，所以只用考虑划分点。由给定条件可选的划分点有 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5

计算每一个划分点下的 $m(s)$，比如对于 $s = 1.5$，$R_{1} = \{1\}, R_{2}=\{2,3,4,5,6,7,8,9,10\}$，$c_{1}=5.56, c_{2}=7.50$，继而可得
$$
m(s)=\min _{j, s}\left[\min _{c_{1}} \sum_{x_{i} \in R_{1}(j, s)}\left(y_{i}-c_{1}\right)^{2}+\min _{c_{2}} \sum_{x_{i} \in R_{2}(j, s)}\left(y_{i}-c_{2}\right)^{2}\right]=0+15.72=15.72
$$
逐个计算后得到以下结果：

| s1   | 1.5   | 2.5   | 3.5  | 4.5  | 5.5  | 6.5  | 7.5  | 8.5   | 9.5   |
| ---- | ----- | ----- | ---- | ---- | ---- | ---- | ---- | ----- | ----- |
| m(s) | 15.72 | 12.07 | 8.36 | 5.78 | 3.91 | 1.93 | 8.01 | 11.73 | 15.74 |

可见当 $s_{1} = 6.5$ 时误差最小，此时 $R_{1}=\{1,2,3,4,5,6\}, R_{2} = \{7,8,9,10\}$

这样就得到了第一个结点
$$
\begin{array}{c}{T_{1}(x)=\left\{\begin{array}{ll}{6.24,} & {x \leqslant 6.5} \\ {8.91,} & {x > 6.5}\end{array}\right.} \end{array}
$$

对 $R_1$ 进一步划分，可选的划分点有 1.5，2.5，3.5，4.5，5.5。得到 $s_{2}$ 及 $m(s)$

| s2   | 1.5  | 2.5  | 3.5  | 4.5  | 5.5  |
| ---- | ---- | ---- | ---- | ---- | ---- |
| m(s) | 1.31 | 0.75 | 0.27 | 0.44 | 1.06 |

可见当 $s_2 = 3.5$ 时误差最小，$R_1$ 被分为 $\{1,2,3\},\{4,5,6\}$ 两部分，得到了第二个结点
$$
\begin{array}{c}{T_{2}(x)=\left\{\begin{array}{ll}{5.72,} & {x \leqslant 3.5} \\ {6.75,} & {x > 3.5}\end{array}\right.} \\ f_{2}(x)={\left\{\begin{array}{ll}{5.72,} & {x \leqslant 3.5} \\ {6.75,} & {3.5 < x \leqslant 6.5} \\ {8.91,} & {x > 6.5}\end{array}\right.}\end{array}
$$
继续迭代
$$
\begin{aligned}
T_{3}(x)&=\left\{\begin{array}{ll}{5.63,} & {x \leqslant 2.5} \\ {5.91,} & {x > 2.5}\end{array} \right. \\
T_{4}(x)&=\left\{\begin{array}{ll}{6.40,} & {x\leqslant4.5} \\ {6.93,} & {x > 4.5}\end{array}\right. \\
T_{5}(x)&=\left\{\begin{array}{ll}{8.80,} & {x \leqslant 8.5} \\ {9.03,} & {x > 8.5}\end{array}\right.
\end{aligned}
$$
回归树生成完毕

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-09-16_regression_tree.png)

此时
$$
L\left(y, f_{5}(x)\right)=\sum_{i=1}^{10}\left(y_{i}-f_{5}\left(x_{i}\right)\right)^{2}=0.13
$$

**5.3 证明CART剪枝算法中，当 $\alpha$ 确定时，存在唯一的最小子树 $T_{\alpha}$ 使损失函数 $C_{\alpha}(T)$ 最小。**

证明：

假设当 $\alpha$ 确定时，决策树 $T_{0}$ 存在两个最小子树 $T_{1}$ 和 $T_{2}$，我们规定 $T_{d}$  是 $T_{2}$ 对 $T_{1}$ 的差异部分。

此时有
$$
C_{\alpha}(T_{1})=C(T_{1})+\alpha|T_{1}| \\
C_{\alpha}(T_{2})=C(T_{2})+\alpha|T_{2}| \\
C_{\alpha}(T_{1})=C_{\alpha}(T_{2})
$$
假设 $T_{d}$ 的根节点为 $t_{d}$,因为 $T_{1}$ 和 $T_{2}$ 是最优子树，则 $ C_{\alpha}(T_{d})=C_{\alpha}(t_{d})$

由此可以将 $T_{1}$ 中的差异部分替换为  $T_{d}$ 得到 $T_{3}$ ，此时
$$
C_{\alpha}(T_{3})=C(T_{3})+\alpha|T_{3}| < C_{\alpha}(T_{1})
$$
与 $T_{1}$ 是最小子树的假设冲突，故当 $\alpha$ 确定时，只存在唯一最小子树

**5.4 证明CART剪枝算法中求出的子树序列 $\{T_{0},T_{1},\ldots,T_{n}\}$ 分别是区间 $\alpha \in [\alpha_{i}, \alpha_{i+1})$ 的最优子树 $T_{\alpha}$，这里 $i=0,1,\ldots,n,\quad 0=\alpha_{0} <\alpha_{1} < \alpha_{2} < \dots < \alpha_{n} < +\infty$ **

证明：

假设在区间 $\alpha \in [\alpha_{i}, \alpha_{i+1})$ 最优子树不是 $T_{\alpha}$ 而是另一棵子树 $T_{t}$，前者对后者的差异部分为 $T_{d}$。此时有
$$
g(t)=\min \alpha=\frac{C(t)-C\left(T_{t}\right)}{\left|T_{t}\right|-1}
$$
将 $T_{t}$ 中的差异部分替换为 $T_{d}$ 得到子树 $T_{m}$,因为 $T_{\alpha}$ 是由 CART 算法剪枝所得，故 $T_{d}$ 内必然包含若干叶子节点，又因为最优子树序列是嵌套的，同时
$$
C(T)=-\sum_{t=1}^{|T|} \sum_{k=1}^{K} N_{t k} \log \frac{N_{t k}}{N_{t}}
$$
所以 $g(t) > g(m)$，与 $\alpha$ 为当前最小值的前提相悖，故在区间 $\alpha \in [\alpha_{i}, \alpha_{i+1})$ 不存在比 $T_{\alpha}$ 更优的子树

---

作者：Daniel Meng

GitHub: [LibertyDream](https://github.com/LibertyDream)

博客：[明月轩](https://libertydream.github.io/)